# NNNN-NN-NN-00-multiple-timeseries-vs-iid-distribution

Quick Note(s)
- For speed, I used ChatGPT to design the simulation, based on some quick instructions that I wanted 
    - a Matern 5/2 covariance function.
    - unique hyperparmater values for each time series (signal mean, signal variance, signal noise, and volatilty)   
- This isn't how I would have written the sim (for better or worse)...but it made the timeseries which is good enough to illustrate the point we want to make.
- ...and by "not how I would have written it" I mean, "Argh, this is painfully bulky"...but still it gets the job done & it's totally stand-alone so not worth the time to fix for a quick demonstration of the concept.

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt


In [ ]:


def matern52_cov(t, tau2=1.0, lam=1.0):
    """
    Matérn 5/2 with rate λ (not length-scale).
    k(r) = τ² * (1 + a + a²/3) * exp(-a), where a = λ * r
    If you prefer length-scale ℓ, set λ = sqrt(5) / ℓ.
    """
    t = np.asarray(t, dtype=float).reshape(-1, 1)
    d = cdist(t, t, metric='euclidean')
    a = lam * d
    K = tau2 * (1.0 + a + (a**2)/3.0) * np.exp(-a)
    return K

def sample_gp_matern52(
    n_time=1000,
    n_series=50,
    mu=0.0,           # scalar or array length n_series
    tau2=1.0,         # scalar or array length n_series  (signal variance)
    sigma=0.1,        # scalar or array length n_series  (obs noise std)
    lam=1.0,          # scalar or array length n_series  (volatility rate)
    t=None,
    jitter=1e-8,
    seed=0
):
    """
    Returns: pandas.DataFrame of shape (n_time, n_series), columns S00..S{n_series-1}
    Each column y_t = f_t + ε_t with f ~ GP(μ, k_M52(τ², λ)) and ε_t ~ N(0, σ²).
    """
    rng = np.random.default_rng(seed)
    if t is None:
        t = np.arange(n_time, dtype=float)
    t = np.asarray(t, dtype=float)

    # Broadcast series-wise hyperparams
    def to_arr(x):
        x = np.asarray(x)
        if x.ndim == 0:
            x = np.repeat(x, n_series)
        assert len(x) == n_series, "Per-series hyperparameter length must equal n_series"
        return x

    mu_arr   = to_arr(mu)
    tau2_arr = to_arr(tau2)
    sigma_arr= to_arr(sigma)
    lam_arr  = to_arr(lam)

    Y = np.empty((n_time, n_series))
    I = np.eye(n_time)

    for j in range(n_series):
        K = matern52_cov(t, tau2=tau2_arr[j], lam=lam_arr[j])
        # Sample latent f via Cholesky
        L = np.linalg.cholesky(K + jitter * I)
        f = mu_arr[j] + L @ rng.normal(size=n_time)
        y = f + rng.normal(scale=sigma_arr[j], size=n_time)
        Y[:, j] = y

    cols = [f"S{j:02d}" for j in range(n_series)]
    return pd.DataFrame(Y, index=np.arange(n_time), columns=cols)



In [ ]:
# # --- Example: wide patient heterogeneity ---
# # Means spread out, variances differ, noise differs, and volatility spans smooth→rough.
# n_time, n_series = 1000, 50
# rng = np.random.default_rng(42)
# mu     = rng.normal(loc=75.0,  scale=10.0,  size=n_series)     # different baselines per patient
# tau2   = rng.lognormal(mean=-0.2, sigma=0.5, size=n_series)  # signal variance
# sigma  = rng.lognormal(mean=-1.2, sigma=0.4, size=n_series)  # obs noise std
# lam    = rng.lognormal(mean=-2.0, sigma=0.6, size=n_series)  # volatility (higher = rougher)

# df_timeseries = sample_gp_matern52(n_time, n_series, mu=mu, tau2=tau2, sigma=sigma, lam=lam, seed=123)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm

# ---- Store sampling parameters (same ones you described) ----
mu_params    = {"loc": 75.0, "scale": 10.0}   # Normal(loc, scale)
tau2_params  = {"mean": 1, "sigma": 1}   # LogNormal: ln(X) ~ N(mean, sigma)
sigma_params = {"mean": 0, "sigma": 1}   # LogNormal: ln(X) ~ N(mean, sigma)
lam_params   = {"mean": 1, "sigma": 1}   # LogNormal: ln(X) ~ N(mean, sigma)

# NOTE: scipy.stats.lognorm uses shape s = sigma and scale = exp(mean).
def lognorm_rvs_pdf_params(p):
    return dict(s=p["sigma"], scale=np.exp(p["mean"]))

# ---- Draw samples for your 50 series USING THE SAME PARAMS AS PDFs ----
n_time, n_series = 1000, 50
rng = np.random.default_rng(42)

mu   = norm.rvs(size=n_series, random_state=rng, **mu_params)
tau2 = lognorm.rvs(size=n_series, random_state=rng, **lognorm_rvs_pdf_params(tau2_params))
sigma= lognorm.rvs(size=n_series, random_state=rng, **lognorm_rvs_pdf_params(sigma_params))
lam  = lognorm.rvs(size=n_series, random_state=rng, **lognorm_rvs_pdf_params(lam_params))

# (unchanged) simulate the GP series
df_timeseries = sample_gp_matern52(n_time, n_series, mu=mu, tau2=tau2, sigma=sigma, lam=lam, seed=123)






# Plots

In [ ]:
"""
Plot the sampling parameters to check you're getting the distributions you want
"""

# ---- Plot PDFs in a 2x2 grid using scipy.stats ----
# Grids: normal over ±4σ, lognormals from near-0 to exp(mean+4*sigma)

x_mu    = np.linspace(mu_params["loc"] - 4*mu_params["scale"],
                      mu_params["loc"] + 4*mu_params["scale"], 1000)
def xgrid_lognorm(p, n=1200):
    return np.linspace(1e-6, np.exp(p["mean"] + 4*p["sigma"]), n)

x_tau2  = xgrid_lognorm(tau2_params)
x_sigma = xgrid_lognorm(sigma_params)
x_lam   = xgrid_lognorm(lam_params)

fig, axes = plt.subplots(2, 2, figsize=(10, 8))

ax = axes[0,0]
ax.plot(x_mu, norm.pdf(x_mu, **mu_params))
ax.set_title(f"mu ~ Normal(loc={mu_params['loc']}, scale={mu_params['scale']})")
ax.set_xlabel("mu"); ax.set_ylabel("PDF"); ax.grid(True, ls=":", alpha=0.4)

ax = axes[0,1]
ax.plot(x_tau2, lognorm.pdf(x_tau2, **lognorm_rvs_pdf_params(tau2_params)))
ax.set_title(f"tau2 ~ LogNormal(mean={tau2_params['mean']}, sigma={tau2_params['sigma']})")
ax.set_xlabel("tau2"); ax.set_ylabel("PDF"); ax.grid(True, ls=":", alpha=0.4)

ax = axes[1,0]
ax.plot(x_sigma, lognorm.pdf(x_sigma, **lognorm_rvs_pdf_params(sigma_params)))
ax.set_title(f"sigma ~ LogNormal(mean={sigma_params['mean']}, sigma={sigma_params['sigma']})")
ax.set_xlabel("sigma"); ax.set_ylabel("PDF"); ax.grid(True, ls=":", alpha=0.4)

ax = axes[1,1]
ax.plot(x_lam, lognorm.pdf(x_lam, **lognorm_rvs_pdf_params(lam_params)))
ax.set_title(f"lam ~ LogNormal(mean={lam_params['mean']}, sigma={lam_params['sigma']})")
ax.set_xlabel("lam"); ax.set_ylabel("PDF"); ax.grid(True, ls=":", alpha=0.4)

fig.tight_layout()

In [ ]:
plt.figure(figsize=(10,5))
order = df_timeseries.median().sort_values().index  # use ascending medians
for i, col in enumerate(order, start=1):
    plt.plot([i]*len(df_timeseries), df_timeseries[col].values, '.b', ms=2, alpha=0.25)

plt.xticks(range(1, len(order)+1), order, rotation=90)
plt.xlim(0.5, len(order)+0.5)
plt.xlabel("Series"); plt.ylabel("Value"); plt.grid(axis='y', ls=':', alpha=0.4)


In [ ]:
df_timeseries

In [ ]:
plt.plot( df.index , df["S00"] , ".")

In [ ]:
plt.plot( df.index , df["S39"] , ".")